In [1]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_regression

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
%run ../common_code.ipynb

In [5]:
#https://www.kaggle.com/datasets/camnugent/california-housing-prices
data = pd.read_csv("../../datasets/housing.csv")

n = data.shape[0]
target = "median_house_value"
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']
data = data[features+[target]]
data.dropna(subset=['total_bedrooms'], inplace=True)
n_features = len(features)
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0


In [6]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [36]:
data_corr = add_extra_features(data, features, extra_size=0.75, mode='prod')

In [37]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42)

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42)

features_corr = list(data_corr.columns.difference([target]))

In [38]:
features_corr

['extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_15',
 'extra_16',
 'extra_17',
 'extra_18',
 'extra_19',
 'extra_2',
 'extra_20',
 'extra_21',
 'extra_22',
 'extra_23',
 'extra_24',
 'extra_3',
 'extra_4',
 'extra_5',
 'extra_6',
 'extra_7',
 'extra_8',
 'extra_9',
 'households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'population',
 'total_bedrooms',
 'total_rooms']

In [39]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    regr = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    regr.fit(data_dict['train'][features], data_dict['train'][target])
    return score_regr(regr, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [40]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [43]:
best_base_params

{'n_estimators': 89, 'max_depth': 8, 'learning_rate': 0.1868954485445952}

In [44]:
model_corr = GradientBoostingRegressor(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='ape_error', task='regr')

In [45]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["ape_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,longitude_0,longitude_1,longitude_2,latitude_0,latitude_1,latitude_2,housing_median_age_0,housing_median_age_1,housing_median_age_2,total_rooms_0,...,extra_22_2,extra_23_0,extra_23_1,extra_23_2,extra_24_0,extra_24_1,extra_24_2,ape_error_0,ape_error_1,ape_error_2
0,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1


In [47]:
disc_data_corr['ape_error'].value_counts()

ape_error
1    1042
2     782
0     782
Name: count, dtype: int64

In [50]:
feat_to_grad_corr = {}
for feat in features_corr+['ape_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'mape': lambda x, y: m.mean_absolute_percentage_error(x,y)*100,
    'r2_score': lambda x, y: m.r2_score(x, y)*100
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'ape_error', categories_corr, score=BicScore)

2025-04-18 22:55:25,730 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,733 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,734 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,735 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,737 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,738 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,739 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,740 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,742 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-18 22:55:25,743 | ERROR    | GraphUtil

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [51]:
gbn_fs.edges

[['longitude_1', 'latitude_0'],
 ['longitude_1', 'extra_10_2'],
 ['longitude_2', 'latitude_0'],
 ['latitude_0', 'extra_10_0'],
 ['latitude_0', 'extra_10_1'],
 ['latitude_0', 'extra_10_2'],
 ['latitude_0', 'extra_4_0'],
 ['latitude_0', 'extra_15_1'],
 ['latitude_0', 'median_income_0'],
 ['latitude_1', 'longitude_0'],
 ['latitude_2', 'extra_10_1'],
 ['housing_median_age_0', 'extra_14_1'],
 ['housing_median_age_0', 'extra_20_0'],
 ['housing_median_age_0', 'extra_16_0'],
 ['housing_median_age_0', 'longitude_1'],
 ['housing_median_age_0', 'extra_5_2'],
 ['housing_median_age_0', 'latitude_0'],
 ['housing_median_age_0', 'extra_11_2'],
 ['housing_median_age_1', 'extra_20_0'],
 ['housing_median_age_2', 'extra_14_1'],
 ['housing_median_age_2', 'extra_14_2'],
 ['housing_median_age_2', 'extra_13_2'],
 ['total_rooms_0', 'extra_22_0'],
 ['total_rooms_0', 'extra_18_0'],
 ['total_rooms_0', 'extra_22_1'],
 ['total_rooms_0', 'extra_1_0'],
 ['total_rooms_1', 'extra_21_1'],
 ['total_rooms_2', 'extra_9_2']

In [52]:
_df=gbn_fs.get_info()
_df[_df['name'].apply(lambda x: 'ape_error' in x.name)]

,name,node_type,data_type,parents,parents_types
30,ape_error_1,Discrete,disc,[extra_4_0],[disc]
68,ape_error_0,Discrete,disc,"[extra_6_0, extra_20_2]","[disc, disc]"
97,ape_error_2,Discrete,disc,"[population_0, extra_20_2]","[disc, disc]"


# Univariate Feature selector

In [53]:
univ_sel = SelectKBest(f_regression, k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [54]:
rfe_sel = RFE(estimator=DecisionTreeRegressor(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [55]:
lsvc = Lasso(alpha=5e3, random_state=42)
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [56]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_15',
 'extra_16',
 'extra_17',
 'extra_18',
 'extra_2',
 'extra_20',
 'extra_21',
 'extra_22',
 'extra_24',
 'extra_3',
 'extra_4',
 'extra_5',
 'extra_6',
 'extra_7',
 'extra_8',
 'extra_9',
 'households',
 'longitude',
 'population',
 'total_bedrooms',
 'total_rooms']

## GBN selector

In [57]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'ape_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 32/32 [00:22<00:00,  1.41it/s]


## GBN selector run

In [58]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc', task='regr')

 22%|██▏       | 7/32 [16:22<58:28, 140.35s/it]  


KeyboardInterrupt: 

In [26]:
gbn_res.head()

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.865,81.97,0,0
1,15,latitude,24.276,72.711,35.886,-11.295
2,14,longitude,28.711,67.151,60.713,-18.079
3,13,total_rooms,28.487,67.259,59.456,-17.947
4,12,extra_2,32.737,62.736,83.245,-23.464


In [27]:
get_best_record(gbn_res, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.865,81.97,0,0


In [28]:
get_best_record(gbn_res, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.865,81.97,0,0


In [29]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc', task='regr')
df_corr_excluded_asc_univ.head()

100%|██████████| 16/16 [04:54<00:00, 18.43s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.914,81.89,0,0
1,15,median_income,18.415,81.261,2.797,-0.768
2,14,extra_6,19.005,80.365,6.092,-1.862
3,13,extra_7,18.052,82.018,0.772,0.157
4,12,extra_3,17.95,82.176,0.201,0.35


In [30]:
get_best_record(df_corr_excluded_asc_univ, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.914,81.89,0,0


In [31]:
get_best_record(df_corr_excluded_asc_univ, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
4,12,extra_3,17.95,82.176,0.201,0.35


In [32]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc', task='regr')
df_corr_excluded_rfe.head()

100%|██████████| 16/16 [07:05<00:00, 26.61s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,16,-,17.831,81.936,0,0
1,15,total_bedrooms,17.802,82.258,-0.162,0.393
2,14,extra_8,18.098,82.146,1.496,0.256
3,13,total_rooms,18.079,82.096,1.389,0.196
4,12,extra_5,18.102,82.16,1.521,0.273


In [33]:
get_best_record(df_corr_excluded_rfe, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
1,15,total_bedrooms,17.802,82.258,-0.162,0.393


In [34]:
get_best_record(df_corr_excluded_rfe, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
13,3,extra_6,17.963,82.438,0.739,0.613


In [35]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict, task='regr')

[17.762, 82.363]

In [ ]:
#logging mape (probably woth to be negative mape)
0.5 random, base 18.467, gbn 17.877, univ 18.467, rfe 18.103 , lasso: 17.89
0.75 random, base 19.137, gbn 18.346, univ 19.137, rfe 18.239 , lasso: 18.338
0.5 noisy, base 18.337, gbn 17.927, univ 18.337, rfe  18.109, lasso: 18.221
0.5 prod, base 17.865, gbn 17.865, univ 17.865, rfe 17.802, lasso: 17.762

In [ ]:
# logging r2_score
0.5 random, base 81.675, gbn 82.667, univ 81.675, rfe  82.27, lasso: 82.616 
0.75 random, base 80.482, gbn 81.832, univ 80.619, rfe  82.188, lasso: 81.806
0.5 noisy, base 81.813, gbn 82.479, univ 81.863, rfe  82.047, lasso: 81.88 
0.5 prod, base 81.97, gbn 81.97, univ 82.176, rfe 82.438, lasso: 82.363


In [ ]:
# #logging mape
# 0.5 random, base 17.877, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
# 0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

# 0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
# 0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

# 0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
# 0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
